In [ ]:
########### not needed anymore? 
import error_patterns
import re
from collections import defaultdict

# INFRA_PATTERNS = {
#     "Infrastructure - Docker": [
#         r"=+\n.+\n=+\n+Cannot connect to the Docker daemon .+",
#         r"=+\n.+\n=+\n+Error response .+",
#         r"^.*Error: unable to pull ddad.artifactory.cc.bmwgroup.net.*",
#         r"Error\: error logging into \"ddad\.artifactory\.cc\.bmwgroup\.net\"\: invalid username\/password",
#         r"Error response from daemon(.*)artifactory\.cc\.bmwgroup\.net[^\s]*\s(?P<details>.*)",
#         r"manifest for artifactory\.cc\.bmwgroup\.net\/.*\:latest not found\: manifest unknown\: The named manifest is not known to the registry",
#         r"^Error: Error parsing image configuration: unable to retrieve auth token: invalid username/password: unknown: Bad props auth token(:?).*$",
#     ]
# }


# Define the regex patterns as provided
BUILD_PATTERNS = error_patterns.BUILD_PATTERNS
INFRA_PATTERNS = error_patterns.INFRA_PATTERNS

def compile_patterns(patterns_dict):
    compiled_patterns = {}
    for tool, patterns in patterns_dict.items():
        # If the value is a set, convert it to a list before compiling
        if isinstance(patterns, set):
            patterns = list(patterns)
        
        # If the value is a list, compile the patterns directly
        if isinstance(patterns, list):
            compiled_patterns[tool] = [re.compile(p) for p in patterns]
        # If the value is another dictionary, compile patterns in each sub-dictionary
        elif isinstance(patterns, dict):
            compiled_patterns[tool] = {}
            for error_type, sub_patterns in patterns.items():
                # If the sub_patterns is a set, convert it to a list
                if isinstance(sub_patterns, set):
                    sub_patterns = list(sub_patterns)
                if isinstance(sub_patterns, list):
                    compiled_patterns[tool][error_type] = [re.compile(p) for p in sub_patterns]
                else:
                    raise TypeError(f"Expected a list for error type '{error_type}' in tool '{tool}', got {type(sub_patterns)}")
        else:
            raise TypeError(f"Expected a list, set, or a dict for tool '{tool}', got {type(patterns)}")
    return compiled_patterns

# Compile the regex patterns for efficiency
#compiled_build_patterns = compile_patterns(BUILD_PATTERNS)
# compiled_infra_patterns = compile_patterns(INFRA_PATTERNS)

# compiled_patterns = {
#     "BUILD": compiled_build_patterns,
#     "INFRA": compiled_infra_patterns,
# }

In [13]:
#### use example pattern

import os
import json
from collections import OrderedDict, defaultdict
import error_patterns
import re



INFRA_PATTERNS = error_patterns.INFRA_PATTERNS

def read_cropped_logs(directory_path):
    """
    Read the cropped files and store them in a dictionary with ordered keys.

    :param directory_path: The directory path to the JSON data files.
    :return: A dictionary with filenames as keys and their corresponding JSON content as values.
    """
    data = {}
    key_order = ['id', 'name', 'stderr', 'stdout_lines', 'node', 'msg']
    # Loop through all files in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith('.json'):
            file_path = os.path.join(directory_path, filename)

            with open(file_path, 'r') as file:
                file_content = json.load(file)
                # Create a new ordered dictionary for each JSON object
                ordered_file_content = [
                    OrderedDict((k, content.get(k, None)) for k in key_order)
                    for content in file_content
                ]
                # Use the filename as the key for the outer dictionary
                data[filename] = {index: content for index, content in enumerate(ordered_file_content)}

    return data

# Function to compile the patterns
def compile_patterns(patterns_dict):
    compiled_patterns = defaultdict(list)
    for category, patterns in patterns_dict.items():
        for pattern in patterns:
            compiled_patterns[category].append(re.compile(pattern))
    return compiled_patterns

# Function to label a single log entry
def label_log_entry(log_entry, regex_list):
    for regex in regex_list:
        if 'msg' in log_entry and regex.search(log_entry['msg']):
            return 'Infrastructure - Docker Build Failure'
    print(log_entry["id"], 
          log_entry["name"],
          log_entry["stderr"],
          log_entry["msg"]
         )
    return 'Unknown'

# Compile the patterns
compiled_infra_patterns = compile_patterns(INFRA_PATTERNS)

#USAGE: 
cropped_logs_path = 'preprocessed_logs'
nested_data = read_cropped_logs(cropped_logs_path)

# Extract the first log entry from the nested data
first_file_key = next(iter(nested_data))
first_log_entry = nested_data[first_file_key][0]

# Label the first log entry
# Pass the list of compiled Infrastructure - Docker patterns directly to the label_log_entry function
label = label_log_entry(first_log_entry, compiled_infra_patterns['Infrastructure - Docker'])
print(f"The label for the first log entry is: {label}")

2aa698b8-b2fd-a900-8da6-00000000002d docker-run  non-zero return code
The label for the first log entry is: Unknown


In [ ]:
########### compile known pattern


import error_patterns
import json
import re
from collections import defaultdict

# Define the regex patterns as provided
BUILD_PATTERNS = error_patterns.BUILD_PATTERNS
INFRA_PATTERNS = error_patterns.INFRA_PATTERNS

def compile_patterns(patterns_dict):
    compiled_patterns = {}
    for tool, patterns in patterns_dict.items():
        # If the value is a set, convert it to a list before compiling
        if isinstance(patterns, set):
            patterns = list(patterns)
        
        # If the value is a list, compile the patterns directly
        if isinstance(patterns, list):
            compiled_patterns[tool] = [re.compile(p) for p in patterns]
        # If the value is another dictionary, compile patterns in each sub-dictionary
        elif isinstance(patterns, dict):
            compiled_patterns[tool] = {}
            for error_type, sub_patterns in patterns.items():
                # If the sub_patterns is a set, convert it to a list
                if isinstance(sub_patterns, set):
                    sub_patterns = list(sub_patterns)
                if isinstance(sub_patterns, list):
                    compiled_patterns[tool][error_type] = [re.compile(p) for p in sub_patterns]
                else:
                    raise TypeError(f"Expected a list for error type '{error_type}' in tool '{tool}', got {type(sub_patterns)}")
        else:
            raise TypeError(f"Expected a list, set, or a dict for tool '{tool}', got {type(patterns)}")
    return compiled_patterns

# Compile the regex patterns for efficiency
compiled_build_patterns = compile_patterns(BUILD_PATTERNS)
compiled_infra_patterns = compile_patterns(INFRA_PATTERNS)

compiled_patterns = {
    "BUILD": compiled_build_patterns,
    "INFRA": compiled_infra_patterns,
}

In [ ]:
## use imported patterns



import os
import json
from collections import OrderedDict

def read_cropped_logs(directory_path):
    """
    Read the cropped files and store them in a dictionary with ordered keys.

    :param directory_path: The directory path to the JSON data files.
    :return: A dictionary with filenames as keys and their corresponding JSON content as values.
    """
    data = {}
    key_order = ['id', 'name', 'stderr', 'stdout_lines', 'node', 'msg']
    # Loop through all files in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith('.json'):
            file_path = os.path.join(directory_path, filename)

            with open(file_path, 'r') as file:
                file_content = json.load(file)
                # Create a new ordered dictionary for each JSON object
                ordered_file_content = [
                    OrderedDict((k, content.get(k, None)) for k in key_order)
                    for content in file_content
                ]
                # Use the filename as the key for the outer dictionary
                data[filename] = {index: content for index, content in enumerate(ordered_file_content)}

    return data

def label_logs(nested_logs, compiled_patterns):
    labeled_logs = {}
    for filename, file_content in nested_logs.items():
        labeled_file_logs = []
        for index, log_entry in file_content.items():
            labels = defaultdict(list)
            # Check each category of patterns
            for category, pattern_dict in compiled_patterns.items():
                for tool, regex_list in pattern_dict.items():
                    for regex in regex_list:
                        # Assuming 'message' is the field containing the text
                        if 'message' in log_entry and regex.search(log_entry['message']):
                            labels[category].append(tool)
                            break  # Stop after the first match within the same tool
            # Append labels to log entry
            log_entry['labels'] = dict(labels)
            labeled_file_logs.append(log_entry)
        # Store the labeled logs using the filename as the key
        labeled_logs[filename] = labeled_file_logs
    return labeled_logs

# Usage:
cropped_logs_path = 'preprocessed_logs'
nested_data = read_cropped_logs(cropped_logs_path)

# Label logs
labeled_nested_logs = label_logs(nested_data, compiled_patterns)

# print the labeled logs for the first file in the dataset
first_file_key = next(iter(labeled_nested_logs))
print(json.dumps(labeled_nested_logs[first_file_key], indent=4))

# Usage: 
cropped_logs_path = 'preprocessed_logs'
data = read_cropped_logs(cropped_logs_path)

# Label  logs
# labeled_logs = label_logs(data, compiled_patterns)
# print(labeled_logs[0])
# # Output the labeled data
# with open('labeled_logs.json', 'w') as file:
#     json.dump(labeled_logs, file, indent=4)

# print("Labeling complete. Labeled logs saved to 'labeled_logs.json'.")